***
<font color="#106eb2" size=8>MONTAGEM DOS DADOS</font><p/>
<font color="#106eb2" size=6>Projeto CoViData</font>
***

***
<font color="#106eb2" size=6>Notebook 01B - Montagem dos Dados</font>
***

<font color="#000000" size=4>Raspagem dos dados de Piramide Populacional - Index Mundi</font>

In [1]:
import json
from bs4 import BeautifulSoup
from bs4.element import Script
from urllib.request import urlopen, Request
from urllib.error import URLError, HTTPError
import pandas as pd
import numpy as np


In [2]:
def gera_json_faixa_etaria(conteudo : str) : 
    conteudo = conteudo[conteudo.rfind('var data = [') + len('var data = [') : conteudo.rfind(', ]; var options')]
    conteudo = '[' + conteudo + ']'
    conteudo = conteudo.replace('age', '"idade"')
    conteudo = conteudo.replace('female', '"feminino"')
    conteudo = conteudo.replace('male', '"masculino"')
    conteudo = conteudo.replace("'", '"')
    objeto = json.loads(conteudo)
    return objeto

def gera_dict_faixa_etaria(conteudo : str, pais: str) : 
    json_object = gera_json_faixa_etaria(conteudo) 
    values = {}
    values['PAIS'] = pais
    for item in json_object :
        key = item['idade']
        value = item['feminino'] + item['masculino']
        values[key] = value
    return values

def gera_dict_pais(link : str, pais: str) :
    url = 'https://www.indexmundi.com/' + link
    headers ={'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    bytesHTML = ''
    try :
        request = Request(url,headers = headers)
        response = urlopen(request)
        bytesHTML = response.read()
    except HTTPError as e:
        print(e.status, e.reason)
    except URLError as e:    
        print(e.reason)

    strHTML = bytesHTML.decode('utf-8')    
    strHTML = " ".join(strHTML.split()).replace('> <', '><')

    soup = BeautifulSoup(strHTML, 'html.parser')
    scripts = soup.find_all('script')
    for script in scripts:
        conteudo = script.string
        if(isinstance(conteudo, Script) and conteudo.find('// data must be in a format with age,') > 0) :
           return gera_dict_faixa_etaria(conteudo, pais)


In [3]:
url = 'https://www.indexmundi.com/population-pyramid/'
headers ={'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
bytesHTML = ''
try :
    request = Request(url,headers = headers)
    response = urlopen(request)
    bytesHTML = response.read()
except HTTPError as e:
    print(e.status, e.reason)
except URLError as e:    
    print(e.reason)

strHTML = bytesHTML.decode('utf-8')    
strHTML = " ".join(strHTML.split()).replace('> <', '><')

lista_paises = []

soup = BeautifulSoup(strHTML, 'html.parser')
div = soup.find('div', class_ = "col-md-12")
anchor = div.find_all('a')
for a in anchor :
  lista_paises.append(gera_dict_pais(a['href'], a.get_text().upper()))

dfIndexMundi = pd.DataFrame(lista_paises)
dfIndexMundi.to_csv('data/pop_index_mundi.csv', index=False, sep=',')
dfIndexMundi.shape

(228, 22)

In [4]:
dfIndexMundi

,PAIS,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,...,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100+
0,AFGHANISTAN,5672736,4897366,4314247,3878103,3910611,3178442,2486682,1855834,1586014,...,838803,630704,445808,286601,157984,66960,19523,3456,334,17
1,ALBANIA,196416,181246,163448,211389,261860,286894,272988,219667,173551,...,188690,178512,137018,95854,82111,52677,23381,7618,1766,186
2,ALGERIA,4411359,4453376,3846205,3036987,2949353,3560644,3578817,3519511,3086863,...,1794179,1390423,1005956,689153,425679,286939,167387,61639,14318,2370
3,AMERICAN SAMOA,4479,4573,4673,4938,4041,4328,3025,2619,2717,...,2702,2086,1396,926,523,328,156,62,17,3
4,ANDORRA,3181,3844,4427,4587,4114,3929,3851,5442,7427,...,7521,6106,4717,3801,2690,1679,1130,630,201,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,WEST BANK,355963,342173,325967,309476,292413,262031,212063,164161,147103,...,86827,61794,39114,27848,18775,11306,5747,1856,355,36
224,WESTERN SAHARA,84590,78874,73252,66755,60051,53728,47526,40614,33613,...,19449,14938,11049,7441,4633,2462,908,225,28,2
225,YEMEN,3758893,3795528,3715415,3410349,2994543,2599797,2280596,1950702,1560332,...,664005,543719,374078,247440,144841,73039,29790,8465,1366,111
226,ZAMBIA,3092857,2628833,2248413,1890158,1601246,1334942,1094554,910445,758878,...,295869,227928,151843,114520,71521,39539,13981,3107,367,20
